In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import chisquare

In [5]:
filepath = "../data_preparation/meta/auto_contours_review_processed.xlsx"

In [6]:
patient_list = [1, 3, 6, 8]
all_data = {"bowel_bag": {
							"auto": [],
							"manual": []
},
"bladder": {
							"auto": [],
							"manual": []
},
"hip": {
							"auto": [],
							"manual": []
},
"rectum": {
							"auto": [],
							"manual": []
}
}

for patient_id in patient_list:
	df = pd.read_excel(filepath, sheet_name="Patient{0:03d}".format(patient_id))
	print(len(df))
	all_df.append(df)

df_raw = pd.concat(all_df)
print(df_raw.head())




22
22
22
22
   index  slice_no direction  slice_coordinate  oar1_bowel_bag  oar1_bladder  \
0      1        48         I                81             NaN           NaN   
1      2        52         I                71             NaN           2.0   
2      3        56         I                61             3.0           1.0   
3      4        60         I                51             3.0           2.0   
4      5        64         I                41             3.0           2.0   

   oar1_hip  oar1_rectum  oar2_bowel_bag  oar2_bladder  oar2_hip  oar2_rectum  
0       NaN          NaN             NaN           NaN       NaN          NaN  
1       NaN          NaN             NaN           1.0       NaN          NaN  
2       2.0          NaN             1.0           1.0       1.0          NaN  
3       1.0          1.0             2.0           2.0       1.0          3.0  
4       1.0          1.0             1.0           2.0       2.0          2.0  


In [4]:
df_raw.head()

,index,slice number,direction,slice coordinate,OAR1,Unnamed: 5,Unnamed: 6,Unnamed: 7,OAR2,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,bowel_bag,bladder,hip,rectum,bowel_bag,bladder,hip,rectum,remarks
1,1.0,48.0,I,81.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,2.0,52.0,I,71.0,NaN,2,1,1,NaN,1,NaN,NaN,NaN
3,3.0,56.0,I,61.0,3,1,2,1,1,NaN,1,NaN,autocontour not excluding mesorectum
4,4.0,60.0,I,51.0,3,2,1,1,2,2,1,3,NaN
